## Removing contaminant kmers
- I would like a set of signatures, where host contamination kmers are removed. Host contamination includes both human and pig signatures. 
- Snakefile takes in a list of SRA numbers, merges the fastmultigather output csvs of that SRA xhuman genome and once x pig genome. 
-  This concatenated csv file is used as a picklist using gather against the same SRA, and this outputs the signature with the matching kmers removed and also a csv file that contains information on what % of the metaG was contamination. 


### Get 1 csv for each category
I want a csv with total contamination for each samples group (pig, pig without MAGs, human)
And then i want contamination by source. So each SRA has 2 lines (pig and human), and each sample group has 1 csv with 200 lines then. 

In [ ]:
# after snakefile was done, can i use a python script or something 
# to for each input metaG get a sum of what was found? 
# can i do that in a snakefile? Probably, may also be able to do this in a folder
# have a notebook that does it, can convert.

In [ ]:
# Run snakefile that concatenates fmg output, then uses that as a picklist
# srun snakemake
srun --account=ctbrowngrp -p med2 -J rmhost -t 10:00:00 -c 50 --mem=50gb --pty bash
mamba activate branchwater

snakemake -s remove_contam.smk --use-conda --resources mem_mb=50000 --rerun-triggers mtime \
-c 50 --rerun-incomplete -k -n